<a href="https://colab.research.google.com/github/Jun-Liu-291/10-Animals/blob/master/Animals_10_v3_Pretrain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive/


In [ ]:
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D, BatchNormalization
from keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img
import numpy as np

Using TensorFlow backend.


In [ ]:
import os

BASE_PATH = '/content/drive/My Drive/dataset/tenAnimals'
TRAIN_PATH = os.path.join(BASE_PATH, 'train')
VAL_PATH = os.path.join(BASE_PATH, 'validation')
TEST_PATH = os.path.join(BASE_PATH, 'test')

batch_size = 128
epochs = 200
IMG_HEIGHT = 128
IMG_WIDTH = 128

In [ ]:
train_image_generator = ImageDataGenerator(
    rescale = 1./ 255,
    rotation_range = 30,
    width_shift_range = .15,
    height_shift_range = .15,
    horizontal_flip = True,
    zoom_range = 0.3
)

validation_image_generator = ImageDataGenerator(rescale=1./255)

In [ ]:
train_data_gen = train_image_generator.flow_from_directory(
    batch_size = batch_size,
    directory = TRAIN_PATH,
    shuffle = True,
    target_size = (IMG_HEIGHT, IMG_WIDTH),
    class_mode = 'categorical'
)

validation_data_gen = validation_image_generator.flow_from_directory(
    batch_size = batch_size,
    directory = VAL_PATH,
    shuffle = True,
    target_size = (IMG_HEIGHT, IMG_WIDTH),
    class_mode = 'categorical'
)

Found 10240 images belonging to 10 classes.
Found 1280 images belonging to 10 classes.


In [ ]:
# model architcture
from keras import optimizers

model = Sequential([Conv2D(64, 3, padding = 'same', activation = 'relu', input_shape = (IMG_HEIGHT, IMG_WIDTH, 3)),
                    Conv2D(64, 3, padding = 'same', activation = 'relu'),
                    MaxPooling2D(),
                    Conv2D(128, 3, padding = 'same', activation = 'relu'),
                    Conv2D(128, 3, padding = 'same', activation = 'relu'),
                    MaxPooling2D(),
                    Conv2D(256, 3, padding = 'same', activation = 'relu'),
                    Conv2D(256, 3, padding = 'same', activation = 'relu'),
                    MaxPooling2D(),
                    Conv2D(512, 3, padding = 'same', activation = 'relu'),
                    Conv2D(512, 3, padding = 'same', activation = 'relu'),
                    Conv2D(512, 3, padding = 'same', activation = 'relu'),
                    MaxPooling2D(),
                    Conv2D(1024, 3, padding = 'same', activation = 'relu'),
                    Conv2D(1024, 3, padding = 'same', activation = 'relu'),
                    Conv2D(1024, 3, padding = 'same', activation = 'relu'),
                    MaxPooling2D(),
                    Flatten(),
                    Dropout(0.4),
                    Dense(512, activation='relu'),
                    Dense(512, activation='relu'),
                    Dense(10, activation='softmax')])

# setting the optimizer and learning rate
learning_rate = 1e-4
optimizer = optimizers.Adam(learning_rate = 1e-4)
  
# Compile the model with the optimizer and learning rate specified in hpyarams
model.compile(optimizer = optimizer,
              loss = 'categorical_crossentropy',
              metrics = ['acc'])

from keras.callbacks import ModelCheckpoint
from keras.callbacks import ReduceLROnPlateau
# change callback function
# apply ReduceLRonPlateau for changing LR
reduceLR = ReduceLROnPlateau(
    monitor="loss",
    factor = 0.5,
    patience = 5,
    verbose = 1,
    mode = 'min',
    min_lr = 1e-6)

# save whole model when get a better val accuracy
checkpoint = ModelCheckpoint(
    filepath = '/content/drive/My Drive/dataset/tenAnimals/model/cat&dog_model.{epoch:02d}-{val_loss:.2f}.h5',
    monitor='val_acc',
    save_weights_only = False,
    save_best_only = True
)

my_callbacks = [reduceLR, checkpoint]



In [ ]:
history = model.fit_generator(
    train_data_gen,
    steps_per_epoch=int(10240/batch_size),
    epochs=epochs,
    validation_data=validation_data_gen,
    validation_steps=int(1028/batch_size),
    callbacks = my_callbacks
)

Epoch 1/200
80/80 [==============================] - 114s 1s/step - loss: 2.2994 - acc: 0.1065 - val_loss: 2.3018 - val_acc: 0.1113
Epoch 2/200
80/80 [==============================] - 112s 1s/step - loss: 2.2596 - acc: 0.1389 - val_loss: 2.3048 - val_acc: 0.1709
Epoch 3/200
80/80 [==============================] - 109s 1s/step - loss: 2.1043 - acc: 0.2188 - val_loss: 2.1090 - val_acc: 0.2168
Epoch 4/200
80/80 [==============================] - 110s 1s/step - loss: 2.0094 - acc: 0.2557 - val_loss: 1.9493 - val_acc: 0.2744
Epoch 5/200
80/80 [==============================] - 109s 1s/step - loss: 1.8873 - acc: 0.3060 - val_loss: 1.9256 - val_acc: 0.3330
Epoch 6/200
80/80 [==============================] - 111s 1s/step - loss: 1.7720 - acc: 0.3561 - val_loss: 1.6160 - val_acc: 0.3848
Epoch 7/200
80/80 [==============================] - 110s 1s/step - loss: 1.6834 - acc: 0.3904 - val_loss: 1.6708 - val_acc: 0.4053
Epoch 8/200
80/80 [==============================] - 108s 1s/step - loss: 1.

In [ ]:
import matplotlib.pyplot as plt
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(acc) + 1)
plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()
plt.figure()
plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()
plt.show()

In [ ]:
model.save('/content/drive/My Drive/dataset/tenAnimals/model/10_Animals_1.h5')

In [ ]:
from keras.models import load_model

model = load_model('/content/drive/My Drive/dataset/tenAnimals/model/10AnimalsPretrained.h5')

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_directory(
    TEST_PATH,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=128,
    class_mode='categorical'
)

Found 1280 images belonging to 10 classes.


In [ ]:
model.evaluate_generator(test_generator)

[1.714066505432129, 0.784375011920929]